In [1]:
import pybullet as p
import pybullet_data
import numpy as np
import cv2
import time

physicsClient = p.connect(p.GUI)

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) Graphics (ADL GT2)
GL_VERSION=4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Intel
Renderer = Mesa Intel(R) Graphics (ADL GT2)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# Load the URDF model

In [2]:
quat = p.getQuaternionFromEuler([1.57, 0, 0])
drone_quat = p.getQuaternionFromEuler([1.57, 0, 0])
drone = p.loadURDF('./cf2/cf2.urdf', globalScaling=2, baseOrientation=drone_quat, basePosition=[0, 0, 2])
forest = p.loadURDF("./forest/forest.urdf", basePosition=[0, 0, 0], baseOrientation=quat, useFixedBase=True)
texture_id = p.loadTexture("./forest/forest.png")
p.changeVisualShape(forest, -1, textureUniqueId=texture_id)

ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: urdfdom: no axis element for Joint, defaulting to (1,0,0) axis prop0_joint

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: urdfdom: no axis element for Joint, defaulting to (1,0,0) axis prop1_joint

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: urdfdom: no axis element for Joint, defaulting to (1,0,0) axis prop2_joint

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: urdfdom: no axis element for Joint, defaulting to (1,0,0) axis prop3_joint

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: urdfdom: no axis element for Joint, defaulting to (1,0,0) axis center_of_mass_joint

ven = Intel
Workaround for some crash in the Intel OpenGL dr

# Camera setup

In [3]:
fov = 60  # Field of view in degrees
aspect = 1  # Aspect ratio
near = 0.001  # Near clipping plane
far = 10  # Far clipping plane
width, height = 1920, 1088  # Image size

# Camera position and orientation relative to the drone (looking down)
camera_position = [0, 0, -0.1]  # Slightly below the drone
camera_target = [0, 0, -1]  # Pointing downward

# Camera projection matrix
projection_matrix = p.computeProjectionMatrixFOV(fov, aspect, near, far)

# Simulate and capture video feed

In [4]:
frame_rate = 10  # Frames per second
duration = 10  # Duration in seconds
total_frames = frame_rate * duration

# Initialize VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('drone_camera_feed.avi', fourcc, frame_rate, (width, height))

# Keyboard control setup

In [5]:
keys = {
    'w': [0.1, 0, 0],
    's': [-0.1, 0, 0],
    'a': [0, -0.1, 0],
    'd': [0, 0.1, 0],
    'q': [0, 0, 0.1],
    'e': [0, 0, -0.1]
}

# Main simulation loop

In [6]:
for i in range(total_frames):
    # Get the drone's position and orientation
    drone_position, drone_orientation = p.getBasePositionAndOrientation(drone)
    
    # Check for keyboard input
    keys_pressed = p.getKeyboardEvents()
    for key, movement in keys.items():
        if ord(key) in keys_pressed and keys_pressed[ord(key)] & p.KEY_WAS_TRIGGERED:
            drone_position = [drone_position[j] + movement[j] for j in range(3)]
            p.resetBasePositionAndOrientation(drone, drone_position, drone_orientation)
    
    # Compute camera's view matrix (camera positioned below drone, looking downward)
    view_matrix = p.computeViewMatrix(
        cameraEyePosition=[drone_position[0] + camera_position[0],
                           drone_position[1] + camera_position[1],
                           drone_position[2] + camera_position[2]],
        cameraTargetPosition=[drone_position[0] + camera_target[0],
                              drone_position[1] + camera_target[1],
                              drone_position[2] + camera_target[2]],
        cameraUpVector=[1, 0, 0]  # Assuming the up direction is along the x-axis
    )
    
    # Render the camera images
    images = p.getCameraImage(width, height, view_matrix, projection_matrix)
    
    # Extract the RGB image
    rgba_img = np.reshape(images[2], (height, width, 4))  # RGBA image
    rgb_img = rgba_img[:, :, :3]  # Convert to RGB by ignoring the alpha channel
    
    # Convert RGB to BGR
    bgr_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2BGR)
    
    # Write the frame to the video file
    out.write(bgr_img)
    
    # Add a small delay to control the simulation speed
    time.sleep(1 / frame_rate)

# Release the VideoWriter

In [7]:
out.release()

# Cleanup

In [8]:
p.disconnect()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
